In [ ]:
!unzip /content/drive/MyDrive/FoCus_modified.zip

Archive:  /content/drive/MyDrive/FoCus_modified.zip
   creating: FoCus_modified/
  inflating: FoCus_modified/val_set.json  
  inflating: __MACOSX/FoCus_modified/._val_set.json  
  inflating: FoCus_modified/test_set.json  
  inflating: __MACOSX/FoCus_modified/._test_set.json  
  inflating: FoCus_modified/train_set.json  
  inflating: __MACOSX/FoCus_modified/._train_set.json  


In [ ]:
!pip install -U sentence-transformers rank_bm25 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.1 MB/s eta 0:00:00


In [ ]:
import json
import nltk
from nltk import sent_tokenize
nltk.download('punkt')
import time
from pprint import pprint

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

In [ ]:
dataset = "/content/FoCus_modified"

In [ ]:
with open(f"{dataset}/train_set.json") as f:
  test_data = json.load(f)

In [ ]:
# bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# bi_encoder.max_seq_length = 512     #Truncate long passages to 256 tokens
# top_k = 32                          #Number of passages we want to retrieve with the bi-encoder


# # cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
# cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
utterance = test_data[1]['utterance']
knowledge = "\n".join(test_data[1]['knowledge'])

In [ ]:
len(test_data[1]['knowledge'])

39

In [ ]:
def get_paragraphs(knowledge):
  paragraphs = []
  for paragraph in knowledge.split("\n"):
      if len(paragraph.strip()) > 0:
          paragraphs.append(sent_tokenize(paragraph.strip()))
  return paragraphs

In [ ]:
def get_queries(utterance):
  queries = []
  for i in range(len(utterance)):
    query = utterance[i][f'dialogue{i+1}'][-2]
    queries.append(query)
  return queries

In [ ]:
# paragraphs_ = []
# for paragraph in test_data[1]['knowledge']:
#     if len(paragraph.strip()) > 0:
#         paragraphs_.append(sent_tokenize(paragraph.strip()))
# len(paragraphs_)

39

In [ ]:
paragraphs = get_paragraphs(knowledge)

In [ ]:
def get_passages(paragraphs, window_size=4):
  passages = []
  for paragraph in paragraphs:
      for start_idx in range(0, len(paragraph), window_size):
          end_idx = min(start_idx+window_size, len(paragraph))
          passages.append(" ".join(paragraph[start_idx:end_idx]))
  return passages

In [ ]:
passages = get_passages(paragraphs)
print("Paragraphs: ", len(paragraphs))
print("Sentences: ", sum([len(p) for p in paragraphs]))
print("Passages: ", len(passages))

Paragraphs:  39
Sentences:  171
Passages:  56


In [ ]:
queries = get_queries(utterance)
queries

["I think I've been there before but I don't remember the name of this place.",
 'What kind of a building is that?',
 'Why was it haunted?',
 'Who built this building?',
 'So, what are the stories about this building?',
 'Is this place famous?']

In [ ]:
for i in range(len(utterance)):
  persona = [utterance[i]['persona_candidate'][idx] for idx, val in enumerate(utterance[i]['persona_grounding']) if val == True ]
  query = utterance[i][f"dialogue{i+1}"][-2]
  knowledge_index = utterance[i]['knowledge_answer_index']
  knowledge_candidate = utterance[i]['knowledge_candidates'][knowledge_index]
  print("---------------------------")
  print("Query:", query)
  print("Knowledge Candidate:", knowledge_candidate)
  print("Persona:", persona)
  print("---------------------------")



---------------------------
Query: I think I've been there before but I don't remember the name of this place.
Knowledge Candidate: Chaonei No. 81 (simplified Chinese: 朝内81号; traditional Chinese: 朝內81號; pinyin: Cháo nèi bāshí yī hào, short for simplified Chinese: 朝阳门内大街81号; traditional Chinese: 朝陽門內大街81號; pinyin: Chāoyáng mén nèi dàjiē bāshí yī hào or Chaoyangmen Inner Street No. 81), sometimes referred to as Chaonei Church, is a house located in the Chaoyangmen neighborhood of the Dongcheng District in Beijing, China. It is a brick structure in the French Baroque architectural style built in the early 20th century, with a larger outbuilding. The municipality of Beijing has designated it a historic building.
Persona: ['I would like to go to Beijing.']
---------------------------
---------------------------
Query: What kind of a building is that?
Knowledge Candidate: It is best known for the widespread belief that it is haunted, and it has been described as "Beijing's most celebrated 'h

In [ ]:
# corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
def retrieve_knowledge(query, passages, corpus_embeddings, n_count = 3):
  # corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  question_embedding = question_embedding.cuda()
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]  # Get the hits for the first query

  cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  cross_scores = cross_encoder.predict(cross_inp)

  # Sort results by the cross-encoder scores
  for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]

  hits = sorted(hits, key=lambda x: x['score'], reverse=True)

  hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
  hit_passages = []
  hit_cross_scores = []
  for hit in hits[0:n_count]:
    hit_cross_scores.append(hit['cross-score'])
    hit_passages.append(passages[hit['corpus_id']].replace("\n", " "))

  return hit_cross_scores, hit_passages


In [ ]:
# knowledge

In [ ]:
# hit_cross_scores, hit_passages = retrieve_knowledge(query, passages, corpus_embeddings, n_count=1)
# pprint(query)
# pprint(hit_passages)

In [ ]:
def retrieval_pipeline(query, knowledge, n_count=2):
  paragraphs = get_paragraphs(knowledge)
  passages = get_passages(paragraphs)
  hit_cross_scores, hit_passages = retrieve_knowledge(query, passages, n_count)
  return hit_cross_scores, hit_passages



In [ ]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 1024     #Truncate long passages to 1024 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder
similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
from tqdm import tqdm
# from exceptions import RunTIme

In [ ]:
results = []
# cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
for idx, data in enumerate(tqdm(test_data[:])):
  if idx >=500:
    break
  dialogID = data['dialogID']
  utterance = data['utterance']
  knowledge = "\n".join(data['knowledge'])
  queries = get_queries(utterance)
  paragraphs = get_paragraphs(knowledge)
  passages = get_passages(paragraphs, window_size=2)
  try:
    corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True)
  except Exception as e:
    print(e, f" DialogId: {dialogID}" )
    continue

  for i, query in enumerate(queries):
    knowledge_index = utterance[i]['knowledge_answer_index']
    knowledge_candidate = utterance[i]['knowledge_candidates'][knowledge_index]

    combined_query = query
    hit_scores, hit_passages = retrieve_knowledge(combined_query, passages, corpus_embeddings, n_count=2)
    hit_knowledge = " ".join(hit_passages)
    persona = [utterance[i]['persona_candidate'][idx] for idx, val in enumerate(utterance[i]['persona_grounding']) if val == True ]
    embeddings1 = similarity_model.encode(knowledge_candidate, convert_to_tensor=True)
    embeddings2 = similarity_model.encode(hit_knowledge, convert_to_tensor=True)
    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    results.append({
        "dialogID": dialogID,
        "utterance": i,
        "query": combined_query,
        "hit_knowledge": hit_knowledge,
        "ground_knowledge": knowledge_candidate,
        "ground_persona": persona,
        "similarity_score": cosine_scores[0].item()
    })



  1%|          | 71/10284 [00:58<1:53:42,  1.50it/s]

The size of tensor a (521) must match the size of tensor b (512) at non-singleton dimension 1  DialogId: PLP6JLJ9KHWI


  2%|▏         | 166/10284 [02:06<1:30:51,  1.86it/s]

The size of tensor a (656) must match the size of tensor b (512) at non-singleton dimension 1  DialogId: G7GOCOZDJ39I


  5%|▍         | 500/10284 [05:42<1:51:48,  1.46it/s]


In [ ]:
results[0]

{'dialogID': 'CGHGPTDOA0O7',
 'utterance': 0,
 'query': "I think I've been there before but I don't remember the name of this place.",
 'hit_knowledge': 'Although it is not fully understood how it came to his attention, Rochdale Town Hall was admired by Adolf Hitler. It has been suggested a visit by Hitler in 1912–13 while staying with his half-brother Alois Hitler, Jr. in Liverpool, or military intelligence on Rochdale, or information from Nazi sympathiser William Joyce (who had lived in Oldham), brought the building to his attention. Rochdale Town Hall is a Victorian-era municipal building in Rochdale, Greater Manchester, England. It is "widely recognised as being one of the finest municipal buildings in the country", and is recorded in the National Heritage List for England as a designated Grade\xa0I listed building.',
 'ground_knowledge': 'Rochdale Town Hall is a Victorian-era municipal building in Rochdale, Greater Manchester, England.',
 'ground_persona': [],
 'similarity_score':

In [ ]:
sum([result["similarity_score"] for result in results ])/len(results)

0.6271324882448608

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame.from_dict(results)

In [ ]:
dataframe.head()

,dialogID,utterance,query,hit_knowledge,ground_knowledge,ground_persona,similarity_score
0,CGHGPTDOA0O7,0,I think I've been there before but I don't rem...,Although it is not fully understood how it cam...,Rochdale Town Hall is a Victorian-era municipa...,[],0.564609
1,CGHGPTDOA0O7,1,Where is this place?,Rochdale Town Hall is a Victorian-era municipa...,Rochdale Town Hall is a Victorian-era municipa...,[I would like to go to england.],0.955529
2,CGHGPTDOA0O7,2,When this was inaugurated?,Built in the Gothic Revival style at a cost of...,Built in the Gothic Revival style at a cost of...,[],0.846570
3,CGHGPTDOA0O7,3,Who was the architect at that time?,Alfred Waterhouse was given the task of design...,"The architect, William Henry Crossland, was th...",[I am not an architect.],0.382931
4,CGHGPTDOA0O7,4,What was the height of the tower?,The design of the original tower was more elab...,The design of the original tower was more elab...,[],1.000000


In [ ]:
dataframe.to_csv("train_knowledge.csv", index=False)

In [ ]:
pd.read_csv("/content/train_knowledge.csv").head()

,dialogID,utterance,query,hit_knowledge,ground_knowledge,ground_persona,similarity_score
0,CGHGPTDOA0O7,0,I think I've been there before but I don't rem...,Although it is not fully understood how it cam...,Rochdale Town Hall is a Victorian-era municipa...,[],0.564609
1,CGHGPTDOA0O7,1,Where is this place?,Rochdale Town Hall is a Victorian-era municipa...,Rochdale Town Hall is a Victorian-era municipa...,['I would like to go to england.'],0.955529
2,CGHGPTDOA0O7,2,When this was inaugurated?,Built in the Gothic Revival style at a cost of...,Built in the Gothic Revival style at a cost of...,[],0.846570
3,CGHGPTDOA0O7,3,Who was the architect at that time?,Alfred Waterhouse was given the task of design...,"The architect, William Henry Crossland, was th...",['I am not an architect.'],0.382931
4,CGHGPTDOA0O7,4,What was the height of the tower?,The design of the original tower was more elab...,The design of the original tower was more elab...,[],1.000000


In [ ]:
# !cp /content/val_knowledge.csv /content/drive/MyDrive

In [ ]:
similarity_dataframe = pd.Series(similarity_scores, index=similarity_scores.keys())

In [ ]:
similarity_dataframe.shape

(9035,)

In [ ]:
similarity_dataframe.to_csv("similarity_scores_w1_c2.csv", index=True, header= ["score"])

In [ ]:
sum(similarity_scores.values())/len(similarity_scores.values())

0.6033647572120737

In [ ]:
# for idx, data in enumerate(test_data):
#   print(idx)
#   utterance = data['utterance']
#   knowledge = "\n".join(data['knowledge'])
#   for i in range(len(utterance)):
#     persona = [utterance[i]['persona_candidate'][idx] for idx, val in enumerate(utterance[i]['persona_grounding']) if val == True ]
#     query = utterance[i][f"dialogue{i+1}"][-2]
#     knowledge_index = utterance[i]['knowledge_answer_index']
#     knowledge_candidate = utterance[i]['knowledge_candidates'][knowledge_index]
#     print("---------------------------")
#     print("Query:", query)
#     print("Knowledge Candidate:", knowledge_candidate)
#     print("Persona:", persona)
#     print("---------------------------")
#   if idx >= 2:
#     break


In [ ]:
!pip install datasets

### Multi Choice Dataset

In [ ]:
from datasets import load_dataset

swag = load_dataset("swag", "regular")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/73546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20005 [00:00<?, ? examples/s]

Dataset swag downloaded and prepared to /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swag["train"][0]

{'video-id': 'anetv_jkn6uvmqwh4',
 'fold-ind': '3416',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'gold-source': 'gold',
 'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'label': 0}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]


def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
preprocess_function(example)

ValueError: ignored

In [ ]:
tokenized_swag = swag.map(preprocess_function, batched=True)

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

In [ ]:
tokenized_swag['test']

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20005
    })
})

### Continued